In [4]:
from utils.io import dataframe_viewer, data_merger, na_col_drop, na_line_drop, dble_col_drop, col_ren
import re, os
import numpy as np
import pandas as pd

# Pollutants concentrations standard

In [5]:
tmp_dir='../../CF_data/Result_traitem/Memoris_seafile/'
sheet='donnees'

## Standard values for soil and water (Industrial site)

In [68]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', 
                   sheet_name='Normes')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
#df.drop(columns=df.columns[-3:], inplace=True)

dataframe_viewer(df, rows=5, cols=12)

Rows : 91, columns : 13


interactive(children=(IntSlider(value=5, description='rows', max=91, min=5, readout=False), IntSlider(value=12…

In [69]:
df_s = df.iloc[:, 8:]
df_w = df.iloc[:, 1:6]

In [70]:
col_ren(df_s, 2)
col_ren(df_w, 2)

,NORMES EAUX SOUTERRAINES,col_1,VR,VS,VI
0,NaN,NaN,NaN,NaN,NaN
1,DECRET SOL,NaN,NaN,NaN,NaN
2,METAUX LOURDS,NaN,NaN,NaN,NaN
3,Arsenic,µg/l,1,10,40
4,Cadmium,µg/l,0.25,5,20
...,...,...,...,...,...
85,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN


In [71]:
for d in [df_s, df_w]:
    d.rename(columns={'col_1':'Unite', 'NORMES SOLS (AFFECTATION INDUSTRIELLE)':'Polluant',
                 'NORMES EAUX SOUTERRAINES':'Polluant'}, inplace=True)
    d.drop(index=[0,1,2], inplace=True)    

In [73]:
df_s = df_s.query('Polluant==Polluant and Unite==Unite and VI==VI')
df_w = df_w.query('Polluant==Polluant and Unite==Unite and VI==VI')
df_s.reset_index(drop=True, inplace=True)
df_w.reset_index(drop=True, inplace=True)

In [60]:
dataframe_viewer(df_s, rows=5, cols=12), dataframe_viewer(df_w, rows=5, cols=12)

Rows : 86, columns : 5


interactive(children=(IntSlider(value=5, description='rows', max=86, min=5, readout=False), IntSlider(value=5,…

Rows : 71, columns : 5


interactive(children=(IntSlider(value=5, description='rows', max=71, min=5, readout=False), IntSlider(value=5,…

(None, None)

In [55]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
df_s.to_csv(tmp_dir+sheet+'_Pol_Std_S_indust.csv', index=False)
df_w.to_csv(tmp_dir+sheet+'_Pol_Std_W_indust.csv', index=False)

## Standard values for soil and water (full)

In [ ]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/Ensemble des résultats Memoris version Seafile.xls', 
                   sheet_name='Données')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
df.drop(columns=df.columns[-3:], inplace=True)

dataframe_viewer(df, rows=5, cols=12)

In [ ]:
df.replace("[v|V]aleur(?:s)? de référence","VR", regex=True, inplace=True)
df.replace("[v|V]aleur(?:s)? seuil","VS", regex=True, inplace=True)
df.replace("[v|V]aleur(?:s)? d'intervention","VI", regex=True, inplace=True)

In [ ]:
df.insert(1, "Groupe", np.nan)
f = None
for i in df.index:
    if not pd.isnull(df.iloc[i, 0]) and pd.isnull(df.iloc[i,2]):
        f = df.iloc[i,0]
    if f is not None: 
        df.loc[i, 'Groupe'] = f

In [ ]:
catg= None
v = None
df.loc[63, :] = np.nan # create a new line at the end
for c in range(len(df.columns)):
    if not pd.isnull(df.iloc[0,c]):
        v = df.iloc[0,c]
    if not pd.isnull(df.iloc[1,c]):
        catg= df.iloc[1,c].split(" ")[0]
        df.iloc[0,c] = v + '_' + catg

In [ ]:
df.iloc[0,0] = "Polluant"
df.iloc[0,1] = "Groupe"
df = col_ren(df, line_to_col=0, mode=0)

In [ ]:
df.drop(index=[0] + list(df.query('Polluant == Groupe').index), inplace=True)
df = df.query('Polluant == Polluant')
df.reset_index(drop=True, inplace=True)

In [ ]:
soil_std = df.iloc[:,range(0,14)]
water_std = df.iloc[:,[0,1]+list(range(14,18))]

In [ ]:
dataframe_viewer(soil_std, rows=5), dataframe_viewer(water_std, rows=5)

In [ ]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
soil_std.to_csv(tmp_dir+sheet+'_Pol_Standards_S.csv', index=False)
water_std.to_csv(tmp_dir+sheet+'_Pol_Standards_W.csv', index=False)

## Standard values for air

In [ ]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/Ensemble des résultats Memoris version Seafile.xls', 
                   sheet_name='Résult air')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5, cols=12)

In [ ]:
df = df.iloc[:, list(range(3,9))][:132]
df.drop(index=[0,1,2]+list(range(4,34)), inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.insert(1, "Groupe", np.nan)
f = None
for i in df.index:
    if not pd.isnull(df.iloc[i, 0]) and pd.isnull(df.iloc[i,2]):
        f = df.iloc[i,0]
    if f is not None: 
        df.loc[i, 'Groupe'] = f

In [ ]:
dataframe_viewer(df, rows=5)